### MNISTダウンロード

In [1]:
import urllib.request

url_base = 'http://yann.lecun.com/exdb/mnist/'
download_file = {
    'train_image':'train-images-idx3-ubyte.gz',
    'train_label':'train-labels-idx1-ubyte.gz',
    'test_image':'t10k-images-idx3-ubyte.gz',
    'test_label':'t10k-labels-idx1-ubyte.gz'
}

In [2]:
dataset_dir = '01.original_data'

for file in download_file.values():
    file_path = dataset_dir + '/' + file
    urllib.request.urlretrieve(url_base + file, file_path)

In [3]:
import gzip

for file in download_file.values():
    gz_file = dataset_dir + '/' + file
    raw_file = dataset_dir + '/' + file.replace('.gz','')
    with gzip.open(gz_file, 'rb') as fp:
        body = fp.read()
        with open(raw_file, 'wb') as w:
            w.write(body)

### 画像ファイル保存

In [4]:
dataset_output_path = '02.data'
train_image_path = dataset_dir + '/' + download_file['train_image'].replace('.gz','')
train_label_path = dataset_dir + '/' + download_file['train_label'].replace('.gz','')
test_image_path = dataset_dir + '/' + download_file['test_image'].replace('.gz','')
test_label_path = dataset_dir + '/' + download_file['test_label'].replace('.gz','')

In [5]:
import struct
import numpy as np
from PIL import Image

def create_mnist_dataset(image_path, label_path, output_path):
    image_file = open(image_path, 'rb')
    magic_number      = struct.unpack('>i', image_file.read(4))[0]
    number_of_images  = struct.unpack('>i', image_file.read(4))[0]
    number_of_rows    = struct.unpack('>i', image_file.read(4))[0]
    number_of_columns = struct.unpack('>i', image_file.read(4))[0]
    bytes_per_image = number_of_rows * number_of_columns 
    format = '%dB' % bytes_per_image
    
    label_file = open(label_path, 'rb')
    magic_number, label_count = struct.unpack('>II', label_file.read(8))
    
                      
    for index in range(0, number_of_images):
        label = struct.unpack('B', label_file.read(1))[0]

        raw_image = image_file.read(bytes_per_image)
        linear_image = struct.unpack(format, raw_image)
        np_array  = np.asarray(linear_image).astype('uint8')
        np_array  = np.reshape(np_array, (28,28),order='C')
        pil_image = Image.fromarray(np_array)
        output_image_path   = output_path + '/' + str(label) + '/' + '%05d.png' % index
        pil_image.save(output_image_path)

In [6]:
import os

for number in range(10):
    os.makedirs(dataset_output_path + '/' + 'train' + '/' + str(number))
    os.makedirs(dataset_output_path + '/' + 'test' + '/' + str(number))

In [7]:
create_mnist_dataset(train_image_path, train_label_path, dataset_output_path + '/' + 'train')
create_mnist_dataset(test_image_path, test_label_path, dataset_output_path + '/' + 'test')